<a href="https://colab.research.google.com/github/qn19325/individualProject/blob/main/conv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

sample_data/
